# Album Colours

installation and importation of required libraries

In [1]:
# pip install opencv-python
# pip install sklearn

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
import sys
import sklearn
import csv
import json

%matplotlib inline

get optimal k with the silhouette method

In [3]:
def get_k_via_silhouette(img, rng=[2,3]):

    K = range(rng[0], rng[1])
    sil_scores = list()
    resized_img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
    # use 1% of original image (to save time)
#     sample_size_to_test = int(img.shape[0]*0.01)
#     print('sample size:', sample_size_to_test)
    resized_img = resized_img.reshape(resized_img.shape[0]*resized_img.shape[1], 3)
    for k in K:
        km = KMeans(n_clusters=k)
        labels = km.fit_predict(resized_img)
        sil_sc = sklearn.metrics.silhouette_score(resized_img, labels, random_state=None)
        sil_scores.append(sil_sc)
    
    optimal_k = K[sil_scores.index(max(sil_scores))]
    return optimal_k

In [4]:
# needs improvment
# bar is added as additional columns for now
# border options?
# add option to include name of colour
def add_colour_bar(img, colours, border=False, extra_width=200):
    combined = np.zeros((img.shape[0], img.shape[1]+extra_width, 3))

    colour_bar = np.zeros((img.shape[0], extra_width, 3), dtype='uint8')

    step = int(og_img.shape[0]/k)
    for c in range(0, len(colours)):
        colour_bar[(c*step):(c+1)*step, :] = colours[c]
    
    if border: 
        border = np.zeros((og_img.shape[0], 2, 3), dtype='uint8')
        combined_img = np.concatenate((og_img, border, colour_bar), axis=1)
    else: 
        combined_img = np.concatenate((og_img, colour_bar), axis=1)
    return combined_img

In [5]:
def rgb_to_hex(rgb):
    rgb = [hex(i)[2:] for i in rgb]
    return '#'+''.join(rgb)

In [6]:
album_data = []
with open('bts_albums.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    headers = next(csvReader, None)
    headers[0] = 'album_title'

    for row in csvReader:
        album_data.append({ headers[i] : row[i] for i in range(0, len(headers)) })
        
# with open('albums_data.json', 'w') as outfile:
#     json.dump(album_data, outfile, indent=4, ensure_ascii=False)# albums_dir = './albums'

albums_dir = './albums'
        
data = []
for album in album_data:
    img_path = albums_dir + '/' + album['img_file']
    og_img = cv2.imread(img_path)
    if og_img is None: continue
    og_img = cv2.cvtColor(og_img, cv2.COLOR_BGR2RGB)
    
    img = og_img.reshape(og_img.shape[0]*og_img.shape[1], 3)
    k = get_k_via_silhouette(og_img)
    
    km = KMeans(n_clusters=k)
    km.fit(img)
    
    num_labels = np.arange(0, len(np.unique(km.labels_)) + 1)
    (hist, _) = np.histogram(km.labels_, bins=num_labels)
    hist = hist.astype("float")
    hist /= hist.sum()
    
    colours = []
    colours_percent = []
    for (percent, color) in zip(hist, km.cluster_centers_):
        colour = color.astype("uint8").tolist()
        colours.append(colour)
        colours_percent.append({'hex': rgb_to_hex(colour), 'percent': float("{0:.2f}".format(percent))})
        
    album['colours'] = colours_percent
#     print(album)
    data.append(album)
    print({'title': '', 'file_name': album, 'k': k, 'colours': colours_percent})
    
    
    colours = [c.astype("uint8").tolist() for c in km.cluster_centers_]
    combined_img = add_colour_bar(og_img, colours)
    cv2.imwrite('./album_colours/' + album['img_file'], cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
    
#     print({'title': '', 'file_name': album, 'k': k, 'colours': colours})

with open('albums_colours_data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4, ensure_ascii=False)# albums_dir = './albums'

{'title': '', 'file_name': {'album_title': 'Map of the Soul: 7', 'alt_title': 'Map of the Soul 7', 'date_kst': '2020-02-21', 'img_file': 'map-of-the-soul-7.jpg', 'type': 'studio', 'colours': [{'hex': '#fefdfb', 'percent': 0.86}, {'hex': '#5193b9', 'percent': 0.14}]}, 'k': 2, 'colours': [{'hex': '#fefdfb', 'percent': 0.86}, {'hex': '#5193b9', 'percent': 0.14}]}
{'title': '', 'file_name': {'album_title': 'Map of the Soul: 7 Black Swan', 'alt_title': 'Map of the Soul 7 Black Swan', 'date_kst': '2020-02-21', 'img_file': 'map-of-the-soul-7-black-swan.jpg', 'type': 'single', 'colours': [{'hex': '#fdfdfe', 'percent': 0.87}, {'hex': '#35405a', 'percent': 0.13}]}, 'k': 2, 'colours': [{'hex': '#fdfdfe', 'percent': 0.87}, {'hex': '#35405a', 'percent': 0.13}]}
{'title': '', 'file_name': {'album_title': 'Map of the Soul: Persona', 'alt_title': 'Map of the Soul Persona', 'date_kst': '2019-04-12', 'img_file': 'map-of-the-soul-persona.jpg', 'type': 'mini', 'colours': [{'hex': '#f67599', 'percent': 0.9